In [1]:
import pandas as pd
import numpy as np
import random

In [8]:
def read_driver_data(filename):
    data = pd.read_csv(filename, delim_whitespace=True, header=None, 
            names=['Year', 'DOY', 'hr', 'min', 'Bx', 'By', 'Bz', 'Vx', 'Vy', 'Vz', 'Np', 'T', 'SYM-H', 'IMF flag', 'SW flag', 'Tilt', 'Pdyn', 'N-index', 'B-index'])
    data['ID'] = data['Year']*10000000 + data['DOY']*10000 + data['hr']*100 + data['min']
    return data

def read_omni_data(filename):
    data = pd.read_csv(filename, delim_whitespace=True, header=None, 
            names=['Year', 'DOY', 'hr', 'B','V', 'phi', 'theta', 'Kp'])
    data['ID'] = data['Year']*100000 + data['DOY']*100 + data['hr']*1
    return data

def compute_N(Vx, Vy, Vz, By, Bz):
    V = np.sqrt(Vx**2 + Vy**2 + Vz**2)
    Bt = np.sqrt(By**2  + Bz**2)
    th = np.arctan2(By, Bz)
    N = 0.86*(V/400)**(4/3) * (Bt/5)**(2/3) * (np.sin(th/2) ** 8) ** (1/3)
    return N.values[0]

def compute_B(Vx, Vy, Vz, By, Bz, Np):
    V = np.sqrt(Vx**2 + Vy**2 + Vz**2)
    Bt = np.sqrt(By**2  + Bz**2)
    th = np.arctan2(By, Bz)
    B = (Np/5)**0.5 * (V / 400)**2.5 * Bt / 5 * np.sin(th/2) ** 6
    return B.values[0]


def input_varying_Vx(driverfilename, omnifilename, timestamp, samplesize):
    data = read_driver_data(driverfilename)
    omni = read_omni_data(omnifilename)
    selectOmni = omni[omni.ID == round(timestamp/100)]
    select = data[data.ID == timestamp]
    
    lists = []
    for i in range(samplesize):
        Vz = 0
        Vy = 0
        Vx = random.uniform(-350,-450)
        
        B = compute_B(Vx, Vy, Vz, select.By, select.Bz, select.Np)

        V = np.sqrt(Vx**2 + Vy**2 + Vz**2)
        Pdyn = select.Np.values[0]/1000000*V**2 * 2

        nv = {'VGSEX' : Vx, 'VGSEY' : Vy, 'VGSEZ' : Vz, 
              'PDYN' : Pdyn, 'Dst': -16.0, 'B0y': select.By.values[0], 
              'B0z': select.Bz.values[0], 'XIND': B, 
              'PS': select.Tilt.values[0], 'IOPT': int(selectOmni.Kp.values[0]/10)+1
             }

        lists.append(nv)

    df = pd.DataFrame(lists)
    df.to_csv('inputfiles/input_Vx')
    return df


def input_varying_B(driverfilename, omnifilename, timestamp, samplesize):
    data = read_driver_data(driverfilename)
    omni = read_omni_data(omnifilename)
    selectOmni = omni[omni.ID == round(timestamp/100)]
    select = data[data.ID == timestamp]
    
    lists = []
    for i in range(samplesize):
        scale = random.uniform(0.75,1.25)
        Bx = select.Bx*scale
        By = select.By*scale
        Bz = select.Bz*scale
        Dst = round(-9*scale*1.8)
        
        Vx = select.Vx
        Vy = select.Vy
        Vz = select.Vz
        
        
        B = compute_B(Vx, Vy, Vz, By, Bz, select.Np)

        V = np.sqrt(Vx.values[0]**2 + Vy.values[0]**2 + Vz.values[0]**2)
        Pdyn = select.Np.values[0]/1000000*V**2 * 2

        nv = {'VGSEX' : Vx.values[0], 'VGSEY' : Vy.values[0], 'VGSEZ' : Vz.values[0], 
              'PDYN' : Pdyn, 'Dst': Dst, 'B0y': By.values[0], 
              'B0z': Bz.values[0], 'XIND': B, 
              'PS': select.Tilt.values[0], 'IOPT': int(selectOmni.Kp.values[0]/10)+1
             }

        lists.append(nv)

    df = pd.DataFrame(lists)
    df.to_csv('inputfiles/input_B')
    return df


input_varying_Vx('data/2004_OMNI_5m_with_TA15_drivers.dat', 'data/omni_20040509_T89.txt', 20041290900, 30)
input_varying_B('data/2004_OMNI_5m_with_TA15_drivers.dat', 'data/omni_20040509_T89.txt', 20041290900, 30)

,VGSEX,VGSEY,VGSEZ,PDYN,Dst,B0y,B0z,XIND,PS,IOPT
0,-469.8,11.1,-28.9,1.582727,-18,-2.193706,-3.577428,0.847076,0.2768,3
1,-469.8,11.1,-28.9,1.582727,-12,-1.462506,-2.385010,0.564731,0.2768,3
2,-469.8,11.1,-28.9,1.582727,-18,-2.209378,-3.602985,0.853127,0.2768,3
3,-469.8,11.1,-28.9,1.582727,-18,-2.182442,-3.559060,0.842727,0.2768,3
4,-469.8,11.1,-28.9,1.582727,-13,-1.521067,-2.480509,0.587343,0.2768,3
5,-469.8,11.1,-28.9,1.582727,-18,-2.180803,-3.556386,0.842094,0.2768,3
6,-469.8,11.1,-28.9,1.582727,-14,-1.726291,-2.815183,0.666589,0.2768,3
7,-469.8,11.1,-28.9,1.582727,-13,-1.604736,-2.616953,0.619651,0.2768,3
8,-469.8,11.1,-28.9,1.582727,-13,-1.618664,-2.639668,0.625030,0.2768,3
9,-469.8,11.1,-28.9,1.582727,-13,-1.597181,-2.604633,0.616734,0.2768,3


In [4]:
def create_reference(datafile, omnifile, timestamp):
    data = read_driver_data(datafile)
    omni = read_omni_data(omnifile)
    selectOmni = omni[omni.ID == round(timestamp/100)]
    select = data[data.ID == timestamp]
    
    Vx = select.Vx
    Vy = select.Vy
    Vz = select.Vz
    
    V = np.sqrt(Vx.values[0]**2 + Vy.values[0]**2 + Vz.values[0]**2)
    
    nv = {'VGSEX' : Vx.values[0], 'VGSEY' : Vy.values[0], 'VGSEZ' : Vz.values[0], 
              'PDYN' : select.Pdyn.values[0], 'Dst': -16.0, 'B0y': select.By.values[0], 
              'B0z': select.Bz.values[0], 'XIND': select['B-index'].values[0], 
              'PS': select.Tilt.values[0], 'IOPT': int(selectOmni.Kp.values[0]/10)+1
             }

    df = pd.DataFrame([nv])
    df.to_csv('inputfiles/reference')
    return df

create_reference('data/2004_OMNI_5m_with_TA15_drivers.dat', 'data/omni_20040509_T89.txt', 20041290900)

,VGSEX,VGSEY,VGSEZ,PDYN,Dst,B0y,B0z,XIND,PS,IOPT
0,-469.8,11.1,-28.9,1.53,-16.0,-1.95,-3.18,0.7774,0.2768,3


In [12]:
data = read_driver_data('data/2004_OMNI_5m_with_TA15_drivers.dat')
select = data[data.ID == 20041290900]
select

,Year,DOY,hr,min,Bx,By,Bz,Vx,Vy,Vz,Np,T,SYM-H,IMF flag,SW flag,Tilt,Pdyn,N-index,B-index,ID
35766,2004,129,9,0,3.13,-1.95,-3.18,-469.8,11.1,-28.9,3.57,74192.0,-9.0,1,2,0.2768,1.53,0.8115,0.7774,20041290900


In [13]:
print('N: ', compute_N(select.Vx, select.Vy, select.Vz, select.By, select.Bz))
print('B: ', compute_B(select.Vx, select.Vy, select.Vz, select.By, select.Bz, select.Np))

N:  0.7937861419041128
B:  0.752971457272638
